#### Making tools

#### Using technique

In [2]:
## Regular Expression

# s.find('substring') return the indice the substring if exists else -1
# @pattern, origin        ----> pattern 
#           baa!|baaa!    ----> baa*!
# '*' : any numbers of the preceding word
# '?' : zero or one word(exist or not exist)
# '.' : any single character
# '^' : the begging of the text
# '$' : the end of the text

def search(pattern, text):
    "Return True if pattern appears anywhere in text"
    if pattern.startswith('^'):
        return match(pattern[1:] , text)
    else:
        return match('.*' + pattern, text)
    
def match(pattern, text):
    "return True if pattern appears at the start of text"
    if pattern = '':
        return True
    
    elif pattern == '$':
        return (text == '')
    elif len(pattern) > 1 and pattern[1] in '*?':
        p, op, pat = pattern[0], pattern[1], pattern[1:]
        if op == '*':
            return match_star(p, pat, text)
        elif op == '?':
            if match1(p, text) and match(pat, text):
                return True
            else:
                return match(pat, text)
        
    else:
        return (match1(pattern[0], text) and match(pattern[1:], text[1:])
def test():
    #test search(pattern, text)
    assert search('baa*!', 'Sheep said baaaa!') == True
    assert search('baa*!', 'Sheep said baaaa humbu!') == False
    assert search('def', 'abcdefg')
    assert search('def$', 'abcdef')
    assert search('def$', 'abcdefg') == False
    assert search('^start', 'not the start') == False
    assert search('start', 'not the start')
    
    # test match(pattern ,text)
    assert match('baa*!', 'Sheep said baaa!') == False
    assert match('baa*!', 'baaaaaaa! said the sheep')
    assert match('start', 'not the start') == False
    assert match('a*b*c*', 'just anything')
    assert mathc('x?', 'text')
    assert match('text?', 'text')
    assert match('text?', 'tex')
    
    assert all(match('aa*bb*cc*$', s) 
               for s in words('abc aaabbcc aaaabcccc'))
    assert not any(match('aa*bb*cc*$', s)
                   for s in words('ac aaabbcccd aaaa-b-cccc'))
    assert all(search('^ab.*aca.*a$', s)
              for s in words('abracadavra abacaa about-acacia-i'))
    assert all(search('t.p', s)
              for s in words('tip top tap atypical tepid stop'))
    assert not any(search('t.p', s)
                  for s in word('TYPE teepee tp'))
    
    return 'test passes'
    
def words(text):return text.split()
                     
                     